# Оптимизированное обучение RecRNN модели

Этот ноутбук использует оптимизированный подход с прогнозированием одного целевого токена.

## 1. Клонирование репозитория

Сначала клонируем репозиторий с кодом и данными:

In [ ]:
!git clone https://your-repo-url.git /kaggle/working/recurrent-pretraining
%cd /kaggle/working/recurrent-pretraining

## 2. Установка зависимостей

Устанавливаем необходимые библиотеки:

In [ ]:
!pip install lightning torch torchdata numpy tqdm

## 3. Настройка директорий данных

Настроим пути к данным и выходным директориям для Kaggle:

In [ ]:
import os
import shutil
from pathlib import Path

# Проверяем, есть ли данные во входном каталоге
input_dir = Path("/kaggle/input")
data_sources = list(input_dir.glob("*/data"))

if not data_sources:
    print("Данные не найдены в /kaggle/input - будем использовать собственные данные из репозитория")
    data_dir = Path("/kaggle/working/recurrent-pretraining/data")
else:
    data_dir = data_sources[0]
    print(f"Будем использовать данные из {data_dir}")

# Создаем символическую ссылку для удобства доступа
working_data_dir = Path("/kaggle/working/data")
if not working_data_dir.exists():
    os.symlink(data_dir, working_data_dir)

# Создаем директории для выходных данных
output_dir = Path("/kaggle/working/output")
output_dir.mkdir(parents=True, exist_ok=True)

print(f"Директория данных: {data_dir}")
print(f"Рабочая ссылка на данные: {working_data_dir}")
print(f"Директория для выходных файлов: {output_dir}")

## 4. Анализ датасета

Запускаем улучшенный анализ данных:

In [ ]:
!python analyze_dataset.py

## 5. Подготовка оптимизированных данных

Подготовим данные, токенизируя их и оптимизируя для предсказания одного токена:

In [ ]:
!python prepare_optimized_data.py

## 6. Обучение с помощью оптимизированного подхода

Запускаем оптимизированный скрипт для Kaggle, который использует одиночные токены для предсказания:

In [ ]:
!python kaggle_run.py --optimize-data --hidden-size 768 --batch-size 16

## 7. Проверка результатов

Проверяем файлы, созданные в процессе обучения:

In [ ]:
import os
print("Files in output directory:")
!ls -la /kaggle/working/output/

## 8. Тестирование обученной модели

Проверяем работу модели на нескольких примерах из тестового набора:

In [ ]:
import json
import torch
from simple_tokenizer import SimpleTokenizer

# Загружаем модель
model_path = "/kaggle/working/output/best_model.pt"
tokenizer = SimpleTokenizer("/kaggle/input/paper-data/data/comparison.1000.12.6/vocab.json")

# Загружаем несколько примеров из тестового набора
with open("/kaggle/input/paper-data/data/comparison.1000.12.6/test.json", "r") as f:
    test_data = json.load(f)[:5]

# Выводим результаты
for item in test_data:
    print(f"Input: {item['input_text']}")
    print(f"Target: {item['target_text']}")
    print("---")

## 9. Сохранение модели для дальнейшего использования

Копируем модель в постоянное хранилище Kaggle:

In [ ]:
!mkdir -p /kaggle/working/saved_models
!cp -r /kaggle/working/output/* /kaggle/working/saved_models/